Преобразование данных модели, полученной интегрированием из центра, в сетку с равномерным шагом по массе. 

In [26]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [27]:
import physics as ph
from Sun import SSM18 as s

In [40]:
def load_model(epoch):
    filename = 'FromCenter/3epoch%.02d.csv' % epoch
    return pd.read_csv(filename)

In [41]:
# load original model
epoch = 0
df = load_model(epoch)
df

,Unnamed: 0,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0,0.000000e+00,0.000000e+00,1.451000e+07,90.073858,0.000000e+00,0.732,1.445415,1.0,1.794120e+17,1.208753e-10
1,1,1.272082e+23,6.960000e+06,1.451000e+07,90.073858,2.198077e+24,0.732,1.445415,0.0,1.794120e+17,1.208753e-10
2,2,1.017666e+24,1.392000e+07,1.450999e+07,90.073846,1.758459e+25,0.732,1.445416,0.0,1.794119e+17,1.208749e-10
3,3,3.434620e+24,2.088000e+07,1.450999e+07,90.073780,5.934786e+25,0.732,1.445417,0.0,1.794117e+17,1.208745e-10
4,4,8.141317e+24,2.784000e+07,1.450997e+07,90.073703,1.406759e+26,0.732,1.445420,0.0,1.794113e+17,1.208738e-10
...,...,...,...,...,...,...,...,...,...,...,...
12122,12122,1.990588e+33,4.711120e+10,6.911885e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,2.000597e+07,3.977335e-48
12123,12123,1.990588e+33,4.711123e+10,6.906126e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.996429e+07,3.937415e-48
12124,12124,1.990588e+33,4.711127e+10,6.900365e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.992266e+07,3.897854e-48
12125,12125,1.990588e+33,4.711130e+10,6.894606e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.988109e+07,3.858666e-48


In [42]:
x = df['Hydrogen'].values
r = df['Radius'].values
m = df['Mass'].values
t = df['Temperature'].values
d = df['Density'].values
l = df['Luminosity'].values
p = df['Pressure'].values
k = df['Opacity'].values
c = df['Convection'].values
v = df['Velocity'].values

In [43]:
# Число точек в исходной R-модели
dimR = len(df.index)
# Число точек в новой M-модели
dimM = 1001

In [44]:
# Шаг по массе
dm = m[-1]/(dimM-1) 

In [45]:
dm

1.990588177082074e+30

In [46]:
# Массивы для данных новой модели
m1 = np.zeros(dimM)
r1 = np.zeros(dimM)
t1 = np.zeros(dimM)
d1 = np.zeros(dimM)
l1 = np.zeros(dimM)
x1 = np.zeros(dimM)
k1 = np.zeros(dimM)
p1 = np.zeros(dimM)
c1 = np.zeros(dimM)
v1 = np.zeros(dimM)

In [47]:
m1[0] = m[0]
r1[0] = r[0]
l1[0] = l[0]
t1[0] = t[0]
d1[0] = d[0]
x1[0] = x[0]
k1[0] = k[0]
p1[0] = p[0]
c1[0] = c[0]
v1[0] = v[0]


In [48]:
j = 1 # index in dimM
i0 = 0
m1[j] = dm
for i in range(1, dimR):
    if m[i] <  m1[j]:
        continue
    f = (m1[j] - m[i-1])/(m[i] - m[i-1])
#     print(i, j, f, i - i0)
    i0 = i
    r1[j] = r[i-1]*(1 - f) + r[i]*f
    t1[j] = t[i-1]*(1 - f) + t[i]*f
    d1[j] = d[i-1]*(1 - f) + d[i]*f
    l1[j] = l[i-1]*(1 - f) + l[i]*f
    x1[j] = x[i-1]*(1 - f) + x[i]*f
    k1[j] = k[i-1]*(1 - f) + k[i]*f
    p1[j] = p[i-1]*(1 - f) + p[i]*f
    c1[j] = c[i-1]*(1 - f) + c[i]*f
    v1[j] = v[i-1]*(1 - f) + v[i]*f
    j += 1
    if j < dimM:
        m1[j] = j*dm
    

In [49]:
df1 = pd.DataFrame({'Mass': m1, 'Radius': r1, 'Temperature': t1, 'Density': d1, 
          'Luminosity': l1, 'Hydrogen': x1, 'Opacity': k1, 'Convection': c1, 'Pressure': p1, 'Velocity': v1})


In [50]:
df1

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.451000e+07,90.073858,0.000000e+00,0.732,1.445415,1.0,1.794120e+17,1.208753e-10
1,1.990588e+30,1.744864e+09,1.439966e+07,89.041127,3.337100e+31,0.732,1.461410,0.0,1.760063e+17,1.149243e-10
2,3.981176e+30,2.201408e+09,1.433513e+07,88.430509,6.557975e+31,0.732,1.470834,0.0,1.740159e+17,1.115997e-10
3,5.971765e+30,2.522902e+09,1.428116e+07,87.917179,9.694220e+31,0.732,1.478767,0.0,1.723544e+17,1.089033e-10
4,7.962353e+30,2.779701e+09,1.423310e+07,87.458197,1.275919e+32,0.732,1.485871,0.0,1.708776e+17,1.065640e-10
...,...,...,...,...,...,...,...,...,...,...
996,1.982626e+33,4.139110e+10,9.072824e+05,0.203440,3.562824e+33,0.732,1.486150,0.0,2.533749e+13,1.087433e-21
997,1.984616e+33,4.188541e+10,8.016714e+05,0.166783,3.562824e+33,0.732,1.191579,0.0,1.835411e+13,2.401102e-22
998,1.986607e+33,4.249698e+10,6.818832e+05,0.125189,3.562824e+33,0.732,0.907523,0.0,1.171825e+13,3.241820e-23
999,1.988598e+33,4.336443e+10,5.328578e+05,0.075302,3.562824e+33,0.732,0.639191,0.0,5.508131e+12,1.428180e-24


In [51]:
_ = df1.to_csv(line_terminator='\n')
filename = "Mass/3epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

Mass/3epoch00.csv
